## Help functions

In [ ]:
def sigmoid(z):
    return 1. / (1 + np.exp(-z))

lam = 1
def gradient(beta, X, y):
    N = y.shape[0]
    temp = (1 - sigmoid(y * np.dot(X, beta))) * (-y)
    return np.dot(X.T, temp) / N + beta / lam

def predict(beta, X):
    y = np.dot(X, beta)
    y[y > 0] = 1
    y[y < 0] = -1
    return y

def zero_one_loss(y_prediction, y_truth):
    return sum(y_prediction != y_truth) / len(y_prediction)
    

## Optimization Methods

In [ ]:
def gradient_descent(X, y, beta_0, tau_0, gamma, m, train = False):
    beta_new = beta_0 - tau_0 * gradient(beta_0, X, y)
    for t in range(1, m):
        tau = tau_0 / (1 + gamma * t)
        beta_new = beta_new - tau * gradient(beta_new, X, y)
        
        if train:
            print(zero_one_loss(predict(beta_new, X), y))
    return beta_new


def stochastic_gradnient(X, y, beta_0, tau_0, gamma, m):
    N = X.shape[0]
    i = random.sample(range(N - 1), 1)
    beta_new = beta_0 - tau_0 * gradient(beta_0, X[i], y[i])
    for t in range(1, m):
        i = random.sample(range(N - 1), 1)
        tau = tau_0 / (1 + gamma * t)
        beta_new = beta_new - tau * gradient(beta_new, X[i], y[i])
        
        print(zero_one_loss(predict(beta_new, X), y))
    return beta_new
    
    
def minibatch_gradnient(X, y, beta_0, tau_0, gamma, m, B):
    N = X.shape[0]
    i = random.sample(range(N - 1), B)
    beta_new = beta_0 - tau_0 * gradient(beta_0, X[i], y[i])
    for t in range(m):
        i = random.sample(range(N - 1), B)
        tau = tau_0 / (1 + gamma * t)
        beta_new = beta_new - tau * gradient(beta_new, X[i], y[i])
        
        print(zero_one_loss(predict(beta_new, X), y))
    return beta_new


def momentum_gradnient(X, y, beta_0, g_0, tau_0, gamma, m, mi):
    N = X.shape[0]
    i = random.sample(range(N - 1), 1)
    g_new = mi * g_0 + (1 - mi) * gradient(beta_0, X[i], y[i])
    beta_new = beta_0 - tau_0 * g_new
    for t in range(m):
        i = random.sample(range(N - 1), 1)
        tau = tau_0 / (1 + gamma * t)
        g_new = mi * g_new + (1 - mi) * gradient(beta_new, X[i], y[i])
        beta_new = beta_new - tau * g_new
        
        print(zero_one_loss(predict(beta_new, X), y))
    return beta_new  


def adam(X, y, beta_0, g_0, q_0, m, mi_1 = 0.9, mi_2 = 0.999, e = 1e-8, tau = 0.0001):
    N = X.shape[0]
    i = random.sample(range(N - 1), 1)
    g_new = mi_1 * g_0 + (1 - mi_1) * gradient(beta_0, X[i], y[i])
    q_new = mi_2 * q_0 + (1 - mi_2) * (gradient(beta_0, X[i], y[i])) ** 2
    
    g_prim = g_new / (1 - mi_1)
    q_prim = q_new / (1 - mi_2)
    
    beta_new = beta_0 - g_prim * tau / (np.sqrt(q_prim) + e)
    
    for t in range(m):
        i = random.sample(range(N - 1), 1)
        
        g_new = mi_1 * g_new + (1 - mi_1) * gradient(beta_new, X[i], y[i])
        q_new = mi_2 * q_new + (1 - mi_2) * (gradient(beta_new, X[i], y[i])) ** 2
        
        g_prim = g_new / (1 - mi_1)
        q_prim = q_new / (1 - mi_2)
    
        beta_new = beta_new - g_prim * tau / (np.sqrt(q_prim) + e)
        
        print(zero_one_loss(predict(beta_new, X), y))
    return beta_new 


def stochastic_average_gradient(X, y, beta, tau, gamma, mi = None, m = 10, eps = 0.000001):
    N = X.shape[0]
    D = X.shape[1]
    g_stored = - y * sigmoid(- y * (np.dot(X, beta))) * X
    g = np.sum(g_stored, axis = 0).reshape(D, 1) / N
    for t in range(m):
        i = random.sample(range(N - 1), 1)
        tau = tau / (1 + gamma * t)
        g_temp = - y[i] * X[i] * sigmoid(- y[i] * np.dot(X[i], beta))
        g = g + (g_temp - g_stored[i]).T / N
        g_stored[i] = g_temp 
        beta = beta * (1 - tau /lam) - tau * g

    return beta


def dual_coordinate_ascent(X, y, beta_0 = None, tau = None, gamma = None, mi = None, m = 10, eps = 0.000001):
    N = X.shape[0]
    alpha = np.random.rand(N)
    alpha = alpha.reshape(-1,1)
    temp = y * alpha
    beta = np.dot(feature_matrix.T, temp) * lam / N
    for t in range(m):
        i = random.sample(range(N - 1), 1)
        print(alpha[i])
        f_prim = y[i] * np.dot(X[i], beta) + np.log(alpha[i]/(1 - alpha[i]))
        f_bis = np.dot(X[i], X[i].T) * lam / N + 1 / (alpha[i] * (1 - alpha[i]))
        
        alpha_old = alpha[i]
        alpha[i] = np.clip(alpha[i] - f_prim / f_bis, eps, 1 - eps)
        
        beta = beta + lam * y[i] * X[i].T * (alpha[i] - alpha_old)
        
    return beta 


def newton_raphson(X, y, beta, tau = None, gamma = None, mi = None, m = 10):
    N = X.shape[0]
    D = X.shape[1]
    for t in range(m):
        scores = np.dot(X, beta)
        labels = y / sigmoid(y * scores)
        temp = (sigmoid(scores) * sigmoid(- scores)).reshape(-1)
        W = np.diag(temp) * lam / N  
        beta = np.linalg.inv(np.identity(D) + X.T @ W @ X) @ X.T @ W @ (scores + labels)
    return beta 